In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import psychoanalyze as pa

In [89]:
min_intensity = -4
max_intensity = 4
k=1
x_0=0
n_trials = 100
n_blocks = 2
n_subjects = 2

In [90]:
intensity_choices = pd.Index(
    range(min_intensity, max_intensity + 1), name="Intensity"
)
intensity_choices

RangeIndex(start=-4, stop=5, step=1, name='Intensity')

In [91]:
trials = pa.sessions.from_frames_dict(
    {
        subj: pa.trials.moc_sample(intensity_choices, n_trials, k, x_0, n_blocks)
        for subj in range(n_subjects)
    }
)
trials


Intensity  Result
Subject Block TrialID                   
0       0     0                3   False
              1                0    True
              2               -3   False
              3               -4   False
              4                3    True
...                          ...     ...
1       1     95               3   False
              96              -2   False
              97              -4   False
              98               3    True
              99              -4   False

[400 rows x 2 columns]

In [95]:
points = trials.groupby(["Subject", "Block"]).apply(pa.points.from_trials)
points

n  Hits  Hit Rate
Subject Block Intensity                    
0       0     -4          9     0  0.000000
              -3         10     1  0.100000
              -2         16     2  0.125000
              -1         15     6  0.400000
               0         17     9  0.529412
               1         10     8  0.800000
               2          4     4  1.000000
               3          8     7  0.875000
               4         11    11  1.000000
        1     -4          9     0  0.000000
              -3         10     0  0.000000
              -2         16     1  0.062500
              -1         15     1  0.066667
               0         17    11  0.647059
               1         10     8  0.800000
               2          4     4  1.000000
               3          8     8  1.000000
               4         11    11  1.000000
1       0     -4         13     0  0.000000
              -3          5     0  0.000000
              -2         11     0  0.000000
              -1         11     2  0.181818
               0         12     7  0.583333
               1         11     8  0.727273
               2         12    11  0.916667
               3         14    14  1.000000
               4         11    11  1.000000
        1     -4         13     0  0.000000
              -3          5     0  0.000000
              -2         11     3  0.272727
              -1         11     3  0.272727
               0         12     3  0.250000
               1         11     7  0.636364
               2         12     9  0.750000
               3         14    11  0.785714
               4         11    11  1.000000

In [114]:
fits = trials.groupby(["Subject", "Block"]).apply(pa.blocks.get_fit)
fits

Subject  Block
0        0        LogisticRegression()
         1        LogisticRegression()
1        0        LogisticRegression()
         1        LogisticRegression()
dtype: object

In [113]:
pa.blocks.make_predictions(fit, intensity_choices=intensity_choices)

Intensity
-4    0.030736
-3    0.074144
-2    0.168215
-1    0.338060
0     0.563266
1     0.765093
2     0.891600
3     0.954068
4     0.981292
Name: Hit Rate, dtype: float64

In [69]:
predictions = pa.subjects.make_predictions(fits, intensity_choices)
predictions

Hit Rate
Block Intensity          
0     -4         0.020199
      -3         0.049223
      -2         0.115058
      -1         0.246149
       0         0.450553
       1         0.673133
       2         0.837972
       3         0.928511
       4         0.970254
1     -4         0.011850
      -3         0.035786
      -2         0.103026
      -1         0.262244
       0         0.523824
       1         0.772956
       2         0.913314
       3         0.970243
       4         0.990187

In [ ]:
predictions = fits.apply(pa.blocks.make_predictions, intensity_choices=intensity_choices)
predictions

In [121]:
params = fits.apply(pa.blocks.fit_params)
params

slope  intercept
Subject Block                     
0       0      0.926379   0.254427
        1      1.599937   0.134766
1       0      1.389645  -0.099479
        1      0.729999  -0.419109

In [59]:
pd.concat(
    {"slope": pd.Series({
        block: fits[block].coef_[0][0] for block in fits
    }),
    "intercept": pd.Series({
        block: fits[block].intercept_[0] for block in fits
    })
    }, names=["param", "Block"])

param      Block
slope      0        1.075417
           1        0.856080
intercept  0       -0.095133
           1        0.045637
dtype: float64